# تحليل بيانات دراسة فرامينغهام لأمراض القلب

هذا الدفتر يعرض الخطوات الرئيسية لتحليل بيانات دراسة فرامينغهام للقلب وبناء نماذج للتنبؤ بخطر الإصابة بأمراض القلب.

## 1. استيراد المكتبات

In [ ]:
# المكتبات الأساسية
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# مكتبات للرسومات البيانية التفاعلية
import plotly.express as px
import plotly.graph_objects as go

# مكتبات التعلم الآلي
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# نماذج التعلم الآلي
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# مقاييس التقييم
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, average_precision_score

# تنسيق الرسومات
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 2. تحميل البيانات

In [ ]:
# تحميل البيانات
df = pd.read_csv('framingham_heart_study.csv')

# عرض الأبعاد
print(f"أبعاد البيانات: {df.shape}")

## 3. استكشاف مبدئي للبيانات

In [ ]:
# عرض أول 5 صفوف
df.head()

In [ ]:
# معلومات عامة عن البيانات
df.info()

In [ ]:
# إحصاءات وصفية
df.describe().T

In [ ]:
# تحليل القيم المفقودة
print("عدد القيم المفقودة في كل عمود:")
print(df.isna().sum())
print(f"\nإجمالي القيم المفقودة: {df.isna().sum().sum()}")

In [ ]:
# توزيع المتغير الهدف
target_counts = df['has_heart_disease'].value_counts()
print("توزيع حالات الإصابة بأمراض القلب:")
print(f"لا يعاني من أمراض القلب (0): {target_counts.get(0, 0)}")
print(f"يعاني من أمراض القلب (1): {target_counts.get(1, 0)}")
print(f"نسبة الإصابة بأمراض القلب: {100 * target_counts.get(1, 0) / df.shape[0]:.2f}%")

## 4. تنظيف البيانات

In [ ]:
# نسخة من البيانات للحفاظ على البيانات الأصلية
df_clean = df.copy()

# تحويل المتغير الهدف إلى int
df_clean['has_heart_disease'] = df_clean['has_heart_disease'].astype(int)

# معالجة القيم المفقودة
# للمتغيرات العددية: استبدال بالوسيط
numeric_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# للمتغيرات الفئوية: استبدال بالقيمة الأكثر شيوعاً
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

# التأكد من أن عمود education هو float إذا وجد
if 'education' in df_clean.columns:
    df_clean['education'] = df_clean['education'].fillna(0).astype(float)

# التحقق من نجاح معالجة القيم المفقودة
print(f"إجمالي القيم المفقودة بعد التنظيف: {df_clean.isna().sum().sum()}")

## 5. تحليل بصري للبيانات

### 5.1 مصفوفة الارتباط (Correlation Heatmap)

In [ ]:
# حساب مصفوفة الارتباط
corr_matrix = df_clean.select_dtypes(include=['float64', 'int64']).corr()

# رسم مصفوفة الارتباط
plt.figure(figsize=(16, 12))
mask = np.triu(np.ones_like(corr_matrix))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', mask=mask, fmt='.2f', linewidths=0.5)
plt.title('مصفوفة الارتباط بين المتغيرات', fontsize=16)
plt.tight_layout()
plt.show()

### 5.2 توزيع المتغيرات (Histograms)

In [ ]:
# اختيار المتغيرات العددية باستثناء المتغير الهدف
numeric_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns.tolist()
if 'has_heart_disease' in numeric_cols:
    numeric_cols.remove('has_heart_disease')

# رسم توزيع كل متغير حسب المتغير الهدف
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 15))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    if i < len(axes):
        for target_val in [0, 1]:
            sns.histplot(
                data=df_clean[df_clean['has_heart_disease'] == target_val],
                x=col,
                bins=15,
                alpha=0.5,
                label=f'Heart Disease: {target_val}',
                ax=axes[i]
            )
        axes[i].set_title(f'توزيع {col}')
        axes[i].legend()

# إزالة المحاور الفارغة
for i in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### 5.3 المخططات الصندوقية (Boxplots)

In [ ]:
# إنشاء مخططات صندوقية للمتغيرات العددية حسب المتغير الهدف
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 15))
axes = axes.flatten()

df_plot = df_clean.copy()
df_plot['has_heart_disease'] = df_plot['has_heart_disease'].map({0: 'لا يوجد مرض قلب', 1: 'يوجد مرض قلب'})

for i, col in enumerate(numeric_cols):
    if i < len(axes):
        sns.boxplot(
            data=df_plot,
            x='has_heart_disease',
            y=col,
            ax=axes[i]
        )
        axes[i].set_title(f'مخطط صندوقي لـ {col}')

# إزالة المحاور الفارغة
for i in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### 5.4 مخطط الأزواج (Pairplot)

In [ ]:
# اختيار أهم المتغيرات فقط للمخطط الزوجي
important_features = ['age', 'sysBP', 'BMI', 'glucose', 'totChol', 'has_heart_disease']
df_important = df_clean[important_features].copy()

# تحويل المتغير الهدف إلى نص للعرض
df_important['has_heart_disease'] = df_important['has_heart_disease'].map({0: 'لا يوجد مرض قلب', 1: 'يوجد مرض قلب'})

# رسم مخطط الأزواج
pairplot = sns.pairplot(
    data=df_important,
    hue='has_heart_disease',
    diag_kind='kde'
)

# إضافة عنوان
pairplot.fig.suptitle('مخطط الأزواج للمتغيرات المهمة', y=1.02, fontsize=16)
plt.show()

## 6. تحضير البيانات للنمذجة

In [ ]:
# فصل المتغيرات المستقلة عن المتغير الهدف
X = df_clean.drop('has_heart_disease', axis=1)
y = df_clean['has_heart_disease']

# تسجيل أسماء المتغيرات للاستخدام لاحقاً
feature_names = X.columns.tolist()

# تقسيم البيانات إلى مجموعات تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"أبعاد بيانات التدريب: {X_train.shape}")
print(f"أبعاد بيانات الاختبار: {X_test.shape}")

# تقييس المتغيرات
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 7. تعريف النماذج

In [ ]:
# تعريف النماذج
models = {
    'الانحدار اللوجستي': LogisticRegression(max_iter=1000, random_state=42),
    'الغابات العشوائية': RandomForestClassifier(n_estimators=100, random_state=42),
    'تعزيز التدرج': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'آلة المتجه الداعم': SVC(probability=True, random_state=42),
    'أقرب الجيران': KNeighborsClassifier(n_neighbors=5),
    'آداببوست': AdaBoostClassifier(n_estimators=100, random_state=42)
}

## 8. تدريب النماذج وتقييم الأداء

In [ ]:
# قاموس لتخزين نتائج النماذج
models_results = {}

# تدريب وتقييم كل نموذج
for name, model in models.items():
    # تدريب النموذج
    model.fit(X_train_scaled, y_train)
    
    # التنبؤ
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # حساب المقاييس
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # حساب منحنى ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    
    # حساب منحنى الدقة-الاسترجاع
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
    avg_precision = average_precision_score(y_test, y_pred_proba)
    
    # حساب مصفوفة الالتباس
    cm = confusion_matrix(y_test, y_pred)
    
    # تخزين النتائج
    models_results[name] = {
        'model': model,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'confusion_matrix': cm,
        'fpr': fpr,
        'tpr': tpr,
        'precision_curve': precision_curve,
        'recall_curve': recall_curve,
        'avg_precision': avg_precision
    }
    
    # طباعة النتائج
    print(f"نموذج: {name}")
    print(f"الدقة: {accuracy:.4f}")
    print(f"الضبط: {precision:.4f}")
    print(f"الاسترجاع: {recall:.4f}")
    print(f"F1: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print("مصفوفة الالتباس:")
    print(cm)
    print("="*50)

## 9. مصفوفات الالتباس للنماذج

In [ ]:
# عرض مصفوفات الالتباس لكل نموذج
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

for i, (name, results) in enumerate(models_results.items()):
    cm = results['confusion_matrix']
    
    # عرض مصفوفة الالتباس
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        cbar=False,
        ax=axes[i]
    )
    
    axes[i].set_title(f"مصفوفة الالتباس - {name}")
    axes[i].set_xlabel('القيمة المتوقعة')
    axes[i].set_ylabel('القيمة الحقيقية')
    axes[i].set_xticklabels(['لا يوجد مرض', 'يوجد مرض'])
    axes[i].set_yticklabels(['لا يوجد مرض', 'يوجد مرض'])

plt.tight_layout()
plt.show()

## 10. مقارنة أداء النماذج

In [ ]:
# استخراج مقاييس الأداء
model_names = list(models_results.keys())
accuracy = [results['accuracy'] for results in models_results.values()]
precision = [results['precision'] for results in models_results.values()]
recall = [results['recall'] for results in models_results.values()]
f1 = [results['f1'] for results in models_results.values()]
auc = [results['auc'] for results in models_results.values()]

# إنشاء مخطط المقارنة
metrics = {
    'النموذج': model_names * 5,
    'المقياس': ['دقة'] * len(model_names) + ['ضبط'] * len(model_names) + ['استرجاع'] * len(model_names) + ['F1'] * len(model_names) + ['AUC'] * len(model_names),
    'القيمة': accuracy + precision + recall + f1 + auc
}

df_metrics = pd.DataFrame(metrics)

plt.figure(figsize=(14, 8))
sns.barplot(x='النموذج', y='القيمة', hue='المقياس', data=df_metrics)
plt.title('مقارنة أداء النماذج', fontsize=16)
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='المقاييس')
plt.tight_layout()
plt.show()

## 11. منحنيات ROC

In [ ]:
# رسم منحنيات ROC
plt.figure(figsize=(12, 8))

# إضافة منحنى الخط المرجعي
plt.plot([0, 1], [0, 1], 'k--', label='عشوائي')

# إضافة منحنى لكل نموذج
for name, results in models_results.items():
    plt.plot(
        results['fpr'],
        results['tpr'],
        label=f"{name} (AUC={results['auc']:.3f})"
    )

plt.xlabel('معدل الإيجابيات الكاذبة')
plt.ylabel('معدل الإيجابيات الحقيقية')
plt.title('منحنيات ROC', fontsize=16)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 12. تحليل أهمية المتغيرات

In [ ]:
# استخراج أهمية المتغيرات من النماذج التي تدعم ذلك
feature_importance = {}

# الغابات العشوائية
if 'الغابات العشوائية' in models_results:
    rf_model = models_results['الغابات العشوائية']['model']
    feature_importance['الغابات العشوائية'] = list(zip(feature_names, rf_model.feature_importances_))

# تعزيز التدرج
if 'تعزيز التدرج' in models_results:
    gb_model = models_results['تعزيز التدرج']['model']
    feature_importance['تعزيز التدرج'] = list(zip(feature_names, gb_model.feature_importances_))

# آداببوست
if 'آداببوست' in models_results:
    ada_model = models_results['آداببوست']['model']
    feature_importance['آداببوست'] = list(zip(feature_names, ada_model.feature_importances_))

# الانحدار اللوجستي
if 'الانحدار اللوجستي' in models_results:
    lr_model = models_results['الانحدار اللوجستي']['model']
    feature_importance['الانحدار اللوجستي'] = list(zip(feature_names, np.abs(lr_model.coef_[0])))

In [ ]:
# عرض أهمية المتغيرات
for model_name, importance in feature_importance.items():
    # ترتيب المتغيرات حسب الأهمية
    sorted_importance = sorted(importance, key=lambda x: x[1], reverse=True)
    
    # استخراج أسماء المتغيرات وقيم الأهمية
    features = [item[0] for item in sorted_importance]
    values = [item[1] for item in sorted_importance]
    
    # رسم مخطط شريطي أفقي
    plt.figure(figsize=(10, 8))
    bars = plt.barh(features, values)
    
    # تنسيق المخطط
    plt.title(f"أهمية المتغيرات - {model_name}", fontsize=16)
    plt.xlabel('الأهمية')
    plt.ylabel('المتغير')
    plt.grid(axis='x', linestyle='--', alpha=0.5)
    
    # إضافة القيم على الأشرطة
    for bar in bars:
        width = bar.get_width()
        plt.text(
            width + 0.01,
            bar.get_y() + bar.get_height() / 2,
            f"{width:.3f}",
            va='center'
        )
    
    plt.tight_layout()
    plt.show()

## 13. الخلاصة

في هذا التحليل، قمنا بما يلي:

1. تحميل واستكشاف بيانات دراسة فرامينغهام للقلب
2. تنظيف البيانات ومعالجة القيم المفقودة
3. إجراء تحليل بصري للبيانات باستخدام مصفوفة الارتباط والمخططات المختلفة
4. تحضير البيانات للنمذجة وتطبيق ستة نماذج مختلفة للتعلم الآلي
5. تقييم أداء النماذج ومقارنتها
6. تحليل أهمية المتغيرات لفهم العوامل الأكثر تأثيراً على خطر الإصابة بأمراض القلب

النماذج أظهرت أداءً جيداً مع اختلافات بسيطة فيما بينها، وتظهر النتائج أن متغيرات مثل العمر وضغط الدم والسكري وتدخين السجائر من أهم العوامل المؤثرة في التنبؤ بمخاطر الإصابة بأمراض القلب.